In [14]:
import pandas as pd
df = pd.read_csv('country_vaccination_stats.csv')
print(df.head())


     country        date  daily_vaccinations   vaccines
0  Argentina  12/29/2020                 NaN  Sputnik V
1  Argentina  12/30/2020             15656.0  Sputnik V
2  Argentina  12/31/2020             15656.0  Sputnik V
3  Argentina    1/1/2021             11070.0  Sputnik V
4  Argentina    1/2/2021              8776.0  Sputnik V


In [15]:
df = df.sort_values(["country", "date"])
# Group the dataframe by country and fill missing values with the minimum daily vaccinations of relevant countries
df["daily_vaccinations"] = df.groupby("country", group_keys=False)["daily_vaccinations"].apply(lambda x: x.fillna(x.min()))
# Fill missing values with 0 (zero) for countries that do not have any valid vaccination number yet
df["daily_vaccinations"] = df.groupby("country", group_keys=False)["daily_vaccinations"].apply(lambda x: x.fillna(0))
df.head()


,country,date,daily_vaccinations,vaccines
3,Argentina,1/1/2021,11070.0,Sputnik V
12,Argentina,1/10/2021,13365.0,Sputnik V
13,Argentina,1/11/2021,14788.0,Sputnik V
14,Argentina,1/12/2021,14056.0,Sputnik V
15,Argentina,1/13/2021,13323.0,Sputnik V


In [16]:
# Group the dataframe by country and calculate the median daily vaccinations for each country
median_daily_vaccinations = df.groupby("country")["daily_vaccinations"].median()
top_3_countries = median_daily_vaccinations.sort_values(ascending=False).head(3)
print(top_3_countries)

country
United States    378253.0
China            276786.0
India            173922.0
Name: daily_vaccinations, dtype: float64


In [17]:
# Filter the dataframe by date 1/6/2021
df_in_01062021 = df[df["date"] == "1/6/2021"]
total_vaccinations = df_01062021["daily_vaccinations"].sum()
print(total_vaccinations)

1485255.0


In [21]:
import sqlite3
df = pd.read_csv("country_vaccination_stats.csv")
conn = sqlite3.connect("country_vaccination_stats.csv.db")
df.to_sql("country_vaccination_stats", conn, if_exists="replace", index=False)
query = '''
UPDATE country_vaccination_stats
SET daily_vaccinations = (
SELECT COALESCE(median_daily_vaccinations, 0)
FROM (
SELECT country, AVG(daily_vaccinations) OVER (PARTITION BY country) AS median_daily_vaccinations
FROM country_vaccination_stats
WHERE daily_vaccinations IS NOT NULL
) AS subquery
WHERE subquery.country = country_vaccination_stats.country
)
WHERE daily_vaccinations IS NULL;
'''
conn.execute(query)
conn.commit()
conn.close()
